imports

In [10]:
import torch
import os
from tqdm.notebook import tqdm as tqdm
import shutil

from HGNN.train.configParser import ConfigParser, getExperimentParamsAndRecord
from HGNN.train import dataLoader

testIndicesFile = "testIndex.csv"
trainingIndicesFile = "trainingIndex.csv"
valIndicesFile = "valIndex.csv"

params

In [11]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data/"
experimentName="biology_paper_augmentation_effect"
trial_hash="825aa4c30a7ace1285f54f2f26af4f9702b5fec2a2f7edf3b35666f7"

output_folder = "/data/BGNN_data/loaders_visualization/biology_paper_augmentation_effect"

cuda=7

cuda

In [12]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 7


load dataset

In [13]:
experimentPathAndName = os.path.join(experimentsPath, experimentName)
experiment_params, experimentRecord = getExperimentParamsAndRecord(experimentsPath, experimentName, trial_hash)
print(experiment_params)

config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName, True)
datasetManager.updateParams(config_parser.fixPaths({**experiment_params,**{'augmented': False}}))
dataset = datasetManager.getDataset()
aug, norm, _ = dataset.toggle_image_loading(augmentation=False, normalization=False)
fineList = dataset.csv_processor.getFineList()
coarseList = dataset.csv_processor.getCoarseList()

{'image_path': 'INHS_cropped', 'suffix': 'biology_paper_200max', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 128, 'n_epochs': 500, 'learning_rate': 5e-05, 'numOfTrials': 3, 'patience': 10, 'fc_width': 200, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.6, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'augmented': True, 'weight_decay': 0.0001, 'img_res': 448, 'tl_freeze': False, 'cnn_layers': 0, 'cnn_channels': 128, 'pretrained': True, 'two_nets': True, 'link_layer': 'layer3', 'dataset_norm': True, 'aug_profile': 'withRotation_PCA1.5'}
Creating dataset...


Loading images: 100%|██████████| 6522/6522 [00:27<00:00, 235.04it/s, fileName=/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588_448.jpg]  


Creating dataset... Done.


visualize loaders

In [49]:
def visualizeLoader(subset, folderName):
    loader = torch.utils.data.DataLoader(subset, batch_size=1)
    with tqdm(total=len(loader), desc="images") as bar:
        for entry in loader: 
            fileName = entry['fileName'][0]
            fine = entry['fine'].item()
            fine_name = str(fine)+"_"+fineList[fine]
            sub_dir = os.path.join(output_folder, fine_name, folderName)
            if not os.path.exists(sub_dir):
                os.makedirs(sub_dir)
            fileFullName = os.path.join(sub_dir,fine_name+"_"+fileName)
            newFile = shutil.copy(os.path.join(dataPath, experiment_params['image_path'], 'images', fileName), fileFullName)
            bar.set_postfix(file=fileName)
            bar.update()


In [50]:
index_file_path = os.path.join(experimentPathAndName, experimentRecord['datasetName'].item(), trainingIndicesFile)
subset = torch.utils.data.Subset(dataset, dataLoader.readFile(index_file_path))

visualizeLoader(subset, 'training_set')

file /home/elhamod/HGNN/experiments/biology_paper_augmentation_effect/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79/trainingIndex.csv read


In [51]:
index_file_path = os.path.join(experimentPathAndName, experimentRecord['datasetName'].item(), valIndicesFile)
subset = torch.utils.data.Subset(dataset, dataLoader.readFile(index_file_path))

visualizeLoader(subset, 'val_set')

file /home/elhamod/HGNN/experiments/biology_paper_augmentation_effect/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79/valIndex.csv read


In [52]:
index_file_path = os.path.join(experimentPathAndName, experimentRecord['datasetName'].item(), testIndicesFile)
subset = torch.utils.data.Subset(dataset, dataLoader.readFile(index_file_path))

visualizeLoader(subset, 'test_set')

file /home/elhamod/HGNN/experiments/biology_paper_augmentation_effect/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79/testIndex.csv read


In [53]:
dataset.toggle_image_loading(augmentation=aug, normalization=norm)

(False, False, True)